In [2]:
import numpy as np
from keras.models import load_model
import os, cv2
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Documents\\iNeuron_intern\\covid_mask'

In [3]:
from covid_md.constant import *
from covid_md.utils import read_yaml, create_directories

In [4]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen= True)
class MaskDetectionConfig:
    root_dir: Path
    face_classifier: Path
    

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_mask_detection_config(self)->MaskDetectionConfig:
        config= self.config.mask_detection
        create_directories([config.root_dir])
        mask_detection_config= MaskDetectionConfig(
            root_dir= config.root_dir,
            face_classifier= config.face_classifier
        )
        return mask_detection_config

In [6]:
class MaskDetection:
    def __init__(self, config: MaskDetectionConfig):
        self.config= config
    def load_model(self):
        path= self.config.root_dir
        os.chdir(path)
        model= str(input("Type Model Name:"))
        model= load_model(model)
        os.chdir("../../")
        return model
    def other_params(self):
        classifier= self.config.face_classifier
        face_classifier= cv2.CascadeClassifier(classifier)
        source= cv2.VideoCapture(0)
        labels_dict= {0: 'Mask', 1: 'No Mask'}
        color_dict= {0: (0,255,0), 1:(0,0,255)}
        return face_classifier, source, labels_dict, color_dict
    def face_mask_detection(seld, model, face_classifier, source, labels_dict, color_dict):
        while(True):
            ret, img= source.read()
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces= face_classifier.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                face_img= gray[y:y+w, x:x+w]
                resize_img= cv2.resize(face_img, (100,100))
                normalized= resize_img/255.0
                reshaped= np.reshape(normalized, (1,100,100,1))
                result= model.predict(reshaped)
                label= np.argmax(result, axis=1)[0]
                cv2.rectangle(img, (x,y), (x+w, y+h), color_dict[label],2)
                cv2.rectangle(img, (x,y-40), (x+w,y), color_dict[label], -1)
                cv2.putText(img, labels_dict[label], (x,y-10), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255,255,255), 2)
            cv2.imshow('LIVE', img)
            key= cv2.waitKey(1)

            if(key=='q'):
                break
        cv2.destroyAllWindows()
        source.release()

In [7]:
try:
    config= ConfigurationManager()
    mask_detection_config= config.get_mask_detection_config()
    mask_detection= MaskDetection(config= mask_detection_config)
    model= mask_detection.load_model()
    face_classifier, source, labels_dict, color_dict= mask_detection.other_params()
    mask_detection.face_mask_detection(model, face_classifier, source, labels_dict, color_dict)
except Exception as e:
    raise e

1/1 [==============================] - 0s 57ms/step


KeyboardInterrupt: 

: 

In [16]:
os.chdir("../../")